#### CNN

In [1]:
import tensorflow as tf
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import cv2
import keras
from tensorflow.keras.metrics import AUC, Precision, Recall, FalseNegatives, FalsePositives,TrueNegatives,TruePositives, CategoricalCrossentropy
from sklearn.metrics import confusion_matrix

In [2]:
#locations of the train and test folder are strored 
train='output/train'
test='output/test'

In [3]:
#preparing data for training and testing on the model

#image data generator allows you to do real time data augmentation 
img_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

#training and testing will hold the augmented and processed images that we will feed to the model 
training= img_datagen.flow_from_directory(train, target_size=(128,128),
                          color_mode='grayscale',batch_size=32, shuffle=True,class_mode='categorical')
testing= img_datagen.flow_from_directory(test, target_size=(128,128),
                          color_mode='grayscale',batch_size=32, shuffle=True,class_mode='categorical')


Found 31500 images belonging to 35 classes.
Found 10500 images belonging to 35 classes.


In [4]:
#creating the model 

from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense , Dropout

In [6]:
#initializing the CNN 

#sequential function provided by keras lets us build a model layer by layer 
model = Sequential()

#convolution layer and pooling 
model.add(Convolution2D(filters=32,kernel_size=3,padding='same',activation='relu',input_shape=(128,128,1)))
#pooling layer helps with downsampling i.e dimensionality reduction
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(filters=32,kernel_size=3,padding='same',activation='relu'))
model.add(Convolution2D(filters=32,kernel_size=3,padding='same',activation='relu'))
#converting the data into a 1D array before inputting to the fully connected layer 
model.add(Flatten())

#connected layer 

model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.40))
model.add(Dense(units=96, activation='relu'))
model.add(Dropout(0.40))
model.add(Dense(units=64, activation='relu'))

model.add(Dense(units=35, activation='softmax')) 

#compiling 
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',AUC(),Precision(),Recall(),FalsePositives(),TrueNegatives(),TruePositives(),FalseNegatives()])

In [7]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 128, 128, 32)      320       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 conv2d_5 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 flatten_1 (Flatten)         (None, 131072)            0         
                                                                 
 dense_4 (Dense)             (None, 128)               16777344  
                                                      

In [8]:
#fitting the model 

model.fit(training, steps_per_epoch=len(training),epochs=5,validation_data=testing,validation_steps=len(testing))

Epoch 1/5
985/985 [==============================] - 614s 622ms/step - loss: 0.3948 - accuracy: 0.8853 - auc: 0.9948 - precision: 0.9491 - recall: 0.8560 - false_positives: 1446.0000 - true_negatives: 1069554.0000 - true_positives: 26964.0000 - false_negatives: 4536.0000 - val_loss: 0.0531 - val_accuracy: 0.9910 - val_auc: 0.9975 - val_precision: 0.9928 - val_recall: 0.9907 - val_false_positives: 75.0000 - val_true_negatives: 356925.0000 - val_true_positives: 10402.0000 - val_false_negatives: 98.0000
Epoch 2/5
985/985 [==============================] - 593s 602ms/step - loss: 0.0733 - accuracy: 0.9772 - auc: 0.9991 - precision: 0.9815 - recall: 0.9731 - false_positives: 579.0000 - true_negatives: 1070421.0000 - true_positives: 30654.0000 - false_negatives: 846.0000 - val_loss: 0.0435 - val_accuracy: 0.9890 - val_auc: 0.9987 - val_precision: 0.9910 - val_recall: 0.9873 - val_false_positives: 94.0000 - val_true_negatives: 356906.0000 - val_true_positives: 10367.0000 - val_false_negatives

In [12]:
model_metrics = model.evaluate(testing,verbose=1)

f1_score = 2*( (model_metrics[3]*model_metrics[4]) / (model_metrics[3]+model_metrics[4]) )

print(f"Accuracy on test set: {round(model_metrics[1]*100,2)}%")
print(f"ROC(Receiver Operation Characteristic) AUC(Area Under Curve): {model_metrics[2]}")
print(f"Precision: {round(model_metrics[3]*100,2)}%")
print(f"Recall: {round(model_metrics[4]*100,2)}%")
print(f"F1-score: {f1_score}")
print(f"Specificity: {(model_metrics[6])/(model_metrics[6]+model_metrics[5])}")

329/329 [==============================] - 43s 131ms/step - loss: 0.0316 - accuracy: 0.9923 - auc: 0.9989 - precision: 0.9928 - recall: 0.9916 - false_positives: 76.0000 - true_negatives: 356924.0000 - true_positives: 10412.0000 - false_negatives: 88.0000
Accuracy on test set: 99.23%
ROC(Receiver Operation Characteristic) AUC(Area Under Curve): 0.9989295601844788
Precision: 99.28%
Recall: 99.16%
F1-score: 0.9921860133602974
Specificity: 0.9997871148459384


In [13]:
print(f"Accuracy on test set: {round(model_metrics[1]*100,2)}%")

Accuracy on test set: 99.23%


In [11]:
model.save('final_model_review.h5')

### Inference 

##### Our model predicts most of the gestures fairly well though it consistently predicts the wrong output for certain letters. Though we've attained a a high accuracy due, to lack of diversity in the training data the model fails to predict the correct sign in some cases. This can be rectified by creating our own dataset with images in different lighting, backgrounds and orientation. 